In [1]:
import xgboost as xgb
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
import time
import datetime

f:\python35\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def abs_MAPE_error(y_pred,y_true):
    return np.mean((np.abs(y_pred-y_true)/np.abs(y_true)) )
def abs_MAPE_error_element(y_pred,y_true):
    return np.abs(y_pred-y_true)/np.abs(y_true)
def abs_error(y_pred,y_true):
    return np.mean((np.abs(y_pred-y_true)))  
def abs_error_element(y_pred,y_true):
    return np.abs(y_pred-y_true)

In [3]:
X = pd.read_csv('C:/Users/Administrator/PycharmProjects/Traffic/Feature/X.csv')
Xtest = pd.read_csv('C:/Users/Administrator/PycharmProjects/Traffic/Feature/Xtest.csv')
Y = pd.read_csv('C:/Users/Administrator/PycharmProjects/Traffic/Feature/Y.csv')

In [4]:
cols_drop = ['TOLLGATE_ID','INTERSECTION_ID','TRN_STA','TRN_END','TST_STA','XTST_END']
X.drop(cols_drop,axis=1,inplace=True)
Xtest.drop(cols_drop,axis=1,inplace=True)

In [ ]:
## #################### # # Feature selection######################################

In [5]:
X.head()

,SA00,SA01,SA02,SA03,SA04,SA05,DAY1MEAN,DAY1MAX,DAY1MIN,DAY1MEDIAN,...,INTERSECTION_ID02,INTERSECTION_ID03,TIME_MEAN,TIME_STD,TIME_MIN,TIME_MAX,TIME_Median,TRN_START_HOUR,TRN_END_HOUR,DAYOFW
0,83.07,65.69,37.85,67.94,52.25,55.26,58.508000,83.48,47.21,50.25,...,0,0,60.343333,15.492514,37.85,83.07,60.475,3,3,4
1,65.69,37.85,67.94,52.25,55.26,45.63,58.678000,83.48,48.06,50.25,...,0,0,54.103333,11.545087,37.85,67.94,53.755,3,3,4
2,37.85,67.94,52.25,55.26,45.63,60.50,56.528000,83.48,39.50,49.89,...,0,0,53.238333,10.664167,37.85,67.94,53.755,3,3,4
3,67.94,52.25,55.26,45.63,60.50,60.63,51.170000,61.71,39.50,49.89,...,0,0,57.035000,7.745581,45.63,67.94,57.880,4,4,4
4,52.25,55.26,45.63,60.50,60.63,65.36,53.171667,63.18,39.50,53.29,...,0,0,56.605000,7.060416,45.63,65.36,57.880,4,4,4


In [6]:
pd.set_option("display.max_columns",100)

In [7]:
cols = list(X.columns)

In [8]:
X.columns

Index(['SA00', 'SA01', 'SA02', 'SA03', 'SA04', 'SA05', 'DAY1MEAN', 'DAY1MAX',
       'DAY1MIN', 'DAY1MEDIAN', 'DAY1STD', 'DAY2MEAN', 'DAY2MAX', 'DAY2MIN',
       'DAY2MEDIAN', 'DAY2STD', 'DAY3MEAN', 'DAY3MAX', 'DAY3MIN', 'DAY3MEDIAN',
       'DAY3STD', 'SUM_LENGTH', 'SI00', 'SI01', 'SI02', 'SI03', 'SI04', 'SI05',
       'SI06', 'SI07', 'SI08', 'FIX', 'RATIO', 'ROUTE_HOUR_head',
       'ROUTE_PCT_head', 'ROUTE_MIN_TTIME', 'ROUTE_MAX_TTIME',
       'ROUTE_LAST_TTIME', 'ROUTE_MEAN_TTIME', 'TSTW00', 'TSTW01', 'TSTW02',
       'TSTW03', 'TSTW04', 'TRNW00', 'TRNW01', 'TRNW02', 'TRNW03', 'TRNW04',
       'TOLLGATE_ID01', 'TOLLGATE_ID02', 'TOLLGATE_ID03', 'INTERSECTION_ID01',
       'INTERSECTION_ID02', 'INTERSECTION_ID03', 'TIME_MEAN', 'TIME_STD',
       'TIME_MIN', 'TIME_MAX', 'TIME_Median', 'TRN_START_HOUR', 'TRN_END_HOUR',
       'DAYOFW'],
      dtype='object')

In [9]:
Xtest.columns

Index(['SA00', 'SA01', 'SA02', 'SA03', 'SA04', 'SA05', 'DAY1MEAN', 'DAY1MAX',
       'DAY1MIN', 'DAY1MEDIAN', 'DAY1STD', 'DAY2MEAN', 'DAY2MAX', 'DAY2MIN',
       'DAY2MEDIAN', 'DAY2STD', 'DAY3MEAN', 'DAY3MAX', 'DAY3MIN', 'DAY3MEDIAN',
       'DAY3STD', 'SUM_LENGTH', 'SI00', 'SI01', 'SI02', 'SI03', 'SI04', 'SI05',
       'SI06', 'SI07', 'SI08', 'FIX', 'RATIO', 'ROUTE_HOUR_head',
       'ROUTE_PCT_head', 'ROUTE_MIN_TTIME', 'ROUTE_MAX_TTIME',
       'ROUTE_LAST_TTIME', 'ROUTE_MEAN_TTIME', 'TSTW00', 'TSTW01', 'TSTW02',
       'TSTW03', 'TSTW04', 'TRNW00', 'TRNW01', 'TRNW02', 'TRNW03', 'TRNW04',
       'TOLLGATE_ID01', 'TOLLGATE_ID02', 'TOLLGATE_ID03', 'INTERSECTION_ID01',
       'INTERSECTION_ID02', 'INTERSECTION_ID03', 'TIME_MEAN', 'TIME_STD',
       'TIME_MIN', 'TIME_MAX', 'TIME_Median', 'TRN_START_HOUR', 'TRN_END_HOUR',
       'DAYOFW'],
      dtype='object')

In [10]:
DEL_COLS = ['TIME_MIN','TIME_MAX','TSTW02','TSTW03','TSTW04','TRNW02','TRNW03','TRNW04'
            ,'ROUTE_MIN_TTIME','ROUTE_MAX_TTIME','ROUTE_LAST_TTIME','ROUTE_MEAN_TTIME','FIX']

In [11]:
for INDEX in DEL_COLS:
    cols.remove(INDEX) 

In [12]:
len(cols)

50

In [68]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2, interaction_only=True, include_bias=False)
X_F = pd.DataFrame(poly.fit_transform(X[cols]))
Xtest_F = pd.DataFrame(poly.transform(Xtest[cols]))
for a in X_F.columns:
    scaler = StandardScaler()
    X_F[a] = scaler.fit_transform(X_F[a])
for a in Xtest_F.columns:
    scaler = StandardScaler()
    Xtest_F[a] = scaler.fit_transform(Xtest_F[a])

f:\python35\lib\site-packages\sklearn\preprocessing\data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
f:\python35\lib\site-packages\sklearn\preprocessing\data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
f:\python35\lib\site-packages\sklearn\preprocessing\data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains

In [14]:
X.fillna(method='ffill',inplace=True)

In [15]:
for a in cols:
    scaler = StandardScaler()
    X[a] = scaler.fit_transform(X[a])
for a in cols:
    scaler = StandardScaler()
    Xtest[a] = scaler.fit_transform(Xtest[a])

f:\python35\lib\site-packages\sklearn\preprocessing\data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
f:\python35\lib\site-packages\sklearn\preprocessing\data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
f:\python35\lib\site-packages\sklearn\preprocessing\data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains

In [16]:
## #################### XGBOOST CV######################################

In [70]:
import xgboost as xgb
depth = 5
eta = 0.03
mcw = 2
params = {"objective": "reg:linear",
          "booster": "gbtree",
          "eta": eta,
          "max_depth": depth,
          "min_child_weight": mcw,
#               "subsample": 0.9,
#               "colsample_bytree": 0.7,
          "silent": 1
          ,"reg_lambda":1
          }
for ind in range(6):
        dtrain = xgb.DMatrix(X_F.values,Y.values[:,ind])
        gbm = xgb.cv(params,dtrain, num_boost_round=1600,nfold=10,early_stopping_rounds=100,verbose_eval=100)

[0]	train-rmse:115.956+0.511617	test-rmse:115.904+4.55239
[100]	train-rmse:31.5926+0.760019	test-rmse:38.4905+8.66657
[200]	train-rmse:26.8603+0.557853	test-rmse:37.8051+8.63046
[300]	train-rmse:24.3378+0.416236	test-rmse:37.7262+8.64938
[400]	train-rmse:22.2898+0.329912	test-rmse:37.7234+8.64733


In [ ]:
## #################### # # ######################################

In [71]:
# 重复划分10次，取平均
# ERROR_LIST=[]
# for x in range(10):
#     ############Split Train DATA and Test Data############################################################
#     from sklearn import cross_validation
#     X_train_m, X_test_m,y_train_m, y_test_m = cross_validation.train_test_split(X,Y, test_size=0.008)
#     ############Split Train DATA and Test Data############################################################

X =  X_F
Y = Y
Xtest = Xtest_F

error_list = []
Ytrain_all = pd.DataFrame()
XGBR = xgb.XGBRegressor(max_depth = 3,learning_rate=0.1,n_estimators=500,reg_alpha=0,reg_lambda=1)
for ind in range(6):
    time1 = time.time()
    XGBR.fit(X.values,Y.values[:,ind] )
    y_true = Y.values[:,ind]
    y_pred = XGBR.predict(X.values)   
    time2 = time.time()  
    print(str(ind)+ '_error:' + str(abs_MAPE_error(y_pred,y_true  ) ) + '__time:'+ str(time2 - time1) )
    error_list.append(abs_MAPE_error(y_pred, y_true  ))

    Ytrain =  XGBR.predict(X.values)
    Ytrain_df = pd.DataFrame(Ytrain)
    Ytrain_all = pd.concat((Ytrain_all,Ytrain_df),axis = 1)   
print(np.mean(error_list))

#%%
Ytrain_true = pd.DataFrame(Y)
Y_error = abs_MAPE_error_element(Ytrain_all.values,Ytrain_true.values  )  
Good_ind =  Y_error.sum(axis=1).argsort()[0:np.int(0.90*len(X))]
#%%
XGBR = xgb.XGBRegressor(max_depth = 5,learning_rate=0.03,n_estimators=1600,reg_alpha=1,reg_lambda=0,min_child_weight=2*20)
Ytest_all = pd.DataFrame()
error_list = []
Ytrain_all = pd.DataFrame()
for ind in range(6):
    time1 = time.time()
    XGBR.fit(X.values[Good_ind],Y.values[Good_ind,ind])
    y_true = Y.values[Good_ind,ind]
    y_pred = XGBR.predict(X.values[Good_ind])
    time2 = time.time()    
    print(str(ind)+ '_error:' + str(abs_MAPE_error(y_pred,y_true  ) ) + '__time:'+ str(time2 - time1) )
    error_list.append(abs_MAPE_error(y_pred, y_true  ))
    Ytest = XGBR.predict(Xtest.values)
    Ytest_df = pd.DataFrame(Ytest)
    Ytest_all = pd.concat((Ytest_all,Ytest_df),axis = 1)    
print(np.mean(error_list))

#     y_test_m.reset_index(drop=True,inplace=True)
#     ERROR_LIST.append(abs_MAPE_error(Ytest_all.values,y_test_m.values))
#     print("train_error:",np.mean(error_list)," vaild_error:",abs_MAPE_error(Ytest_all.values,y_test_m.values))
# print(np.mean(ERROR_LIST))    

0_error:0.153720852509__time:62.44256019592285
1_error:0.155069912942__time:61.45444416999817
2_error:0.156122915495__time:61.06640267372131
3_error:0.157230440812__time:64.45341038703918
4_error:0.158270990726__time:71.0960783958435
5_error:0.158550842053__time:64.6845281124115
0.156494325756
0_error:0.109643586617__time:310.6423690319061
1_error:0.098949731648__time:332.7063624858856
2_error:0.0961827930904__time:334.1346478462219
3_error:0.092968150401__time:322.8992359638214
4_error:0.104854466254__time:332.1648745536804
5_error:0.0980169969469__time:325.0770251750946
0.100102620826


In [81]:
################################GBR############################################################################################
from sklearn.ensemble import GradientBoostingRegressor
X =  X_F
Y = Y
Xtest = Xtest_F

error_list = []
Ytrain_all = pd.DataFrame()
GBR = GradientBoostingRegressor(loss='lad', alpha=0.95,
                                n_estimators=500, max_depth=3,
                                learning_rate=.1)
for ind in range(6):
    time1 = time.time()
    GBR.fit(X.values,Y.values[:,ind] )
    y_true = Y.values[:,ind]
    y_pred = GBR.predict(X.values)   
    time2 = time.time()  
    print(str(ind)+ '_error:' + str(abs_MAPE_error(y_pred,y_true  ) ) + '__time:'+ str(time2 - time1) )
    error_list.append(abs_MAPE_error(y_pred, y_true  ))

    Ytrain =  GBR.predict(X.values)
    Ytrain_df = pd.DataFrame(Ytrain)
    Ytrain_all = pd.concat((Ytrain_all,Ytrain_df),axis = 1)   
print(np.mean(error_list))

#%%
Ytrain_true = pd.DataFrame(Y)
Y_error = abs_MAPE_error_element(Ytrain_all.values,Ytrain_true.values  )  
Good_ind =  Y_error.sum(axis=1).argsort()[0:np.int(0.90*len(X))]
#%%
GBR = GradientBoostingRegressor(loss='lad', max_depth = 5,learning_rate=0.1,n_estimators=500,alpha=0.95)
Ytest_all = pd.DataFrame()
error_list = []
Ytrain_all = pd.DataFrame()
for ind in range(6):
    time1 = time.time()
    GBR.fit(X.values[Good_ind],Y.values[Good_ind,ind])
    y_true = Y.values[Good_ind,ind]
    y_pred = GBR.predict(X.values[Good_ind])
    time2 = time.time()    
    print(str(ind)+ '_error:' + str(abs_MAPE_error(y_pred,y_true  ) ) + '__time:'+ str(time2 - time1) )
    error_list.append(abs_MAPE_error(y_pred, y_true  ))
    Ytest = GBR.predict(Xtest.values)
    Ytest_df = pd.DataFrame(Ytest)
    Ytest_all = pd.concat((Ytest_all,Ytest_df),axis = 1)    
print(np.mean(error_list))


0_error:0.153380017789__time:490.17570996284485
1_error:0.155202661418__time:482.4617567062378
2_error:0.157633182616__time:480.90252685546875
3_error:0.156892300652__time:490.8454329967499
4_error:0.159078831987__time:493.63083362579346
5_error:0.163123027392__time:484.9735424518585
0.157551670309
0_error:0.121049273591__time:758.1156032085419
1_error:0.119879712335__time:755.8619253635406
2_error:0.121162951672__time:782.1280355453491
3_error:0.119638091485__time:755.3660798072815
4_error:0.123031344121__time:743.571275472641
5_error:0.125733187265__time:750.9688549041748
0.121749093411


In [82]:
# Gernerate Final Result

In [83]:
Xtest = pd.read_csv('C:/Users/Administrator/PycharmProjects/Traffic/Feature/Xtest.csv')
Ytest_all.columns = [0,1,2,3,4,5]
result = pd.concat([Ytest_all,Xtest],axis=1)
result = result.loc[:,[0,1,2,3,4,5,'TOLLGATE_ID','INTERSECTION_ID','TRN_STA','TRN_END']]
result['TRN_END'] = result['TRN_STA'].apply(lambda x :
str((datetime.datetime.strptime(x,'%Y-%m-%d %H:%M:%S') + datetime.timedelta(hours=2))))

In [84]:
result

,0,1,2,3,4,5,TOLLGATE_ID,INTERSECTION_ID,TRN_STA,TRN_END
0,107.543006,111.218810,123.975093,83.691566,112.508836,116.852699,2,A,2016-10-18 06:00:00,2016-10-18 08:00:00
1,155.153972,110.766861,127.056763,120.234649,107.772225,117.742982,3,A,2016-10-18 06:00:00,2016-10-18 08:00:00
2,105.274894,82.120826,78.101344,114.361901,96.107879,96.061945,1,B,2016-10-18 06:00:00,2016-10-18 08:00:00
3,96.870006,93.800747,106.802936,108.187701,97.552012,102.797716,3,B,2016-10-18 06:00:00,2016-10-18 08:00:00
4,121.536191,121.324948,107.409717,145.407667,125.440526,122.887880,1,C,2016-10-18 06:00:00,2016-10-18 08:00:00
5,137.583845,144.885204,133.814793,137.833252,146.279653,134.064819,3,C,2016-10-18 06:00:00,2016-10-18 08:00:00
6,92.860936,96.856834,114.912757,85.120151,96.967145,101.593453,2,A,2016-10-19 06:00:00,2016-10-19 08:00:00
7,121.413687,119.711426,131.995699,114.786538,120.159080,107.261639,3,A,2016-10-19 06:00:00,2016-10-19 08:00:00
8,86.032911,87.407678,98.751248,109.218684,90.684427,87.233625,1,B,2016-10-19 06:00:00,2016-10-19 08:00:00
9,96.775891,95.270015,93.282032,103.882560,95.117514,89.367004,3,B,2016-10-19 06:00:00,2016-10-19 08:00:00


In [85]:
sub_result = pd.DataFrame()
for x in range(result.shape[0]):
    t = result.loc[x,:]
    for y in range(6):
        end_time = str(datetime.datetime.strptime(t['TRN_END'],'%Y-%m-%d %H:%M:%S') + datetime.timedelta(minutes=20*(y+1)))
        start_time = str((datetime.datetime.strptime(t['TRN_END'],'%Y-%m-%d %H:%M:%S') + datetime.timedelta(minutes=20*y)))
        timewindow = '['+start_time+','+end_time+')'
        each_result = pd.DataFrame({'intersection_id':[t['INTERSECTION_ID']],'tollgate_id':[t['TOLLGATE_ID']]
        ,'time_window':[timewindow] ,'avg_travel_time':[t[y]]})  
        sub_result = pd.concat((sub_result,each_result),axis=0)

In [86]:
sub_result

,avg_travel_time,intersection_id,time_window,tollgate_id
0,107.543006,A,"[2016-10-18 08:00:00,2016-10-18 08:20:00)",2
0,111.218810,A,"[2016-10-18 08:20:00,2016-10-18 08:40:00)",2
0,123.975093,A,"[2016-10-18 08:40:00,2016-10-18 09:00:00)",2
0,83.691566,A,"[2016-10-18 09:00:00,2016-10-18 09:20:00)",2
0,112.508836,A,"[2016-10-18 09:20:00,2016-10-18 09:40:00)",2
0,116.852699,A,"[2016-10-18 09:40:00,2016-10-18 10:00:00)",2
0,155.153972,A,"[2016-10-18 08:00:00,2016-10-18 08:20:00)",3
0,110.766861,A,"[2016-10-18 08:20:00,2016-10-18 08:40:00)",3
0,127.056763,A,"[2016-10-18 08:40:00,2016-10-18 09:00:00)",3
0,120.234649,A,"[2016-10-18 09:00:00,2016-10-18 09:20:00)",3


In [87]:
cols = ['intersection_id','tollgate_id','time_window','avg_travel_time']
sub_result = sub_result.ix[:, cols]

In [88]:
sub_result.to_csv('GBR1.csv',index=False,)